### A decade of Verified Twitter mentioning @devinnunes

In [183]:
import pandas as pd
import geopandas as gpd
import cpi
from urllib.request import urlopen 
import pyarrow
import jenkspy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import numpy as np
from altair import datum
import altair as alt
alt.renderers.enable('notebook')
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,.0f}'.format

In [184]:
src = pd.read_json(open('/Users/mhustiles/data/github/twint/devinnunes-verified.json', encoding='utf8'),lines=True)

In [185]:
src.loc[2]

cashtags                                                          []
conversation_id                                  1198297653349507072
created_at                                       2019-11-23 17:49:58
date                                             2019-11-23 00:00:00
geo                                                                 
hashtags                                       [#holmes, #fionahill]
id                                               1198297653349507072
likes_count                                                        4
link               https://twitter.com/taradublinrocks/status/119...
mentions                                                [devinnunes]
name                                        Tara Dublin, Fashion Cop
near                                                                
photos                                                            []
place                                                               
quote_url          https://twitter

In [307]:
src['eastern_created_at'] = src['created_at'].dt.tz_localize("GMT").dt.tz_convert('US/Eastern')
src['eastern_date_time'] = pd.to_datetime(src['eastern_created_at']).dt.strftime('%Y/%m/%d %H:%M')
src['eastern_date'] = pd.to_datetime(src['eastern_created_at']).dt.strftime('%Y/%m/%d/')
src['eastern_time'] = src['eastern_created_at'].dt.time
src.eastern_date_time = pd.to_datetime(src.eastern_date_time)
src['eastern_date'] = pd.to_datetime(src.eastern_date)

In [308]:
src['year'] = src['eastern_created_at'].dt.year
src['month'] = src['eastern_created_at'].dt.month
src['day'] = src['eastern_created_at'].dt.day
src['hour'] = src['eastern_created_at'].dt.hour
src['minute'] = src['eastern_created_at'].dt.minute

In [309]:
src[['id', 'user_id', 'year', 'conversation_id']] =\
src[['id', 'user_id', 'year', 'conversation_id']].astype(str)

In [310]:
src.dtypes

cashtags                                  object
conversation_id                           object
created_at                        datetime64[ns]
date                              datetime64[ns]
geo                                       object
hashtags                                  object
id                                        object
likes_count                                int64
link                                      object
mentions                                  object
name                                      object
near                                      object
photos                                    object
place                                     object
quote_url                                 object
replies_count                              int64
reply_to                                  object
retweet                                     bool
retweet_date                              object
retweet_id                                object
retweets_count      

In [311]:
tweets = pd.DataFrame(src)

In [312]:
len(tweets)

22552

In [313]:
impeachment = tweets[(tweets['date'] > '2019-11-12') & (tweets['date'] < '2019-11-23')]

In [314]:
len(impeachment)

2908

In [315]:
impeachment.head()

,cashtags,conversation_id,created_at,date,geo,hashtags,id,likes_count,link,mentions,...,video,eastern_created_at,eastern_date,eastern_time,year,month,day,hour,minute,eastern_date_time
138,[],1198057403545079808,2019-11-23 07:47:42,2019-11-22,,"[#devinnunesisatraitor, #devinnunesgotcaught]",1198146089905340417,32,https://twitter.com/slack2thefuture/status/119...,[devinnunes],...,0,2019-11-23 02:47:42-05:00,2019-11-23,02:47:42,2019,11,23,2,47,2019-11-23 02:47:00
139,[],1198083851484094464,2019-11-23 07:44:24,2019-11-22,,[],1198145255909888000,139,https://twitter.com/natfinnonE/status/11981452...,"[oliverdarcy, thegarance, devinnunes]",...,0,2019-11-23 02:44:24-05:00,2019-11-23,02:44:24,2019,11,23,2,44,2019-11-23 02:44:00
140,[],1198057403545079808,2019-11-23 07:43:13,2019-11-22,,[],1198144957942419456,8,https://twitter.com/dana1981/status/1198144957...,"[devinnunes, gop]",...,1,2019-11-23 02:43:13-05:00,2019-11-23,02:43:13,2019,11,23,2,43,2019-11-23 02:43:00
141,[],1198144318323621888,2019-11-23 07:40:40,2019-11-22,,"[#karma, #devinnunesgotcaught, #devinnunesisat...",1198144318323621888,53,https://twitter.com/taradublinrocks/status/119...,[devinnunes],...,0,2019-11-23 02:40:40-05:00,2019-11-23,02:40:40,2019,11,23,2,40,2019-11-23 02:40:00
142,[],1198138097562611712,2019-11-23 07:36:23,2019-11-22,,[],1198143241431863297,2,https://twitter.com/taradublinrocks/status/119...,"[altamacresists, devinnunes]",...,0,2019-11-23 02:36:23-05:00,2019-11-23,02:36:23,2019,11,23,2,36,2019-11-23 02:36:00


In [316]:
dates = impeachment.groupby(['date', 'hour']).agg('size').reset_index(name='count')

In [317]:
impeachment.head()

,cashtags,conversation_id,created_at,date,geo,hashtags,id,likes_count,link,mentions,...,video,eastern_created_at,eastern_date,eastern_time,year,month,day,hour,minute,eastern_date_time
138,[],1198057403545079808,2019-11-23 07:47:42,2019-11-22,,"[#devinnunesisatraitor, #devinnunesgotcaught]",1198146089905340417,32,https://twitter.com/slack2thefuture/status/119...,[devinnunes],...,0,2019-11-23 02:47:42-05:00,2019-11-23,02:47:42,2019,11,23,2,47,2019-11-23 02:47:00
139,[],1198083851484094464,2019-11-23 07:44:24,2019-11-22,,[],1198145255909888000,139,https://twitter.com/natfinnonE/status/11981452...,"[oliverdarcy, thegarance, devinnunes]",...,0,2019-11-23 02:44:24-05:00,2019-11-23,02:44:24,2019,11,23,2,44,2019-11-23 02:44:00
140,[],1198057403545079808,2019-11-23 07:43:13,2019-11-22,,[],1198144957942419456,8,https://twitter.com/dana1981/status/1198144957...,"[devinnunes, gop]",...,1,2019-11-23 02:43:13-05:00,2019-11-23,02:43:13,2019,11,23,2,43,2019-11-23 02:43:00
141,[],1198144318323621888,2019-11-23 07:40:40,2019-11-22,,"[#karma, #devinnunesgotcaught, #devinnunesisat...",1198144318323621888,53,https://twitter.com/taradublinrocks/status/119...,[devinnunes],...,0,2019-11-23 02:40:40-05:00,2019-11-23,02:40:40,2019,11,23,2,40,2019-11-23 02:40:00
142,[],1198138097562611712,2019-11-23 07:36:23,2019-11-22,,[],1198143241431863297,2,https://twitter.com/taradublinrocks/status/119...,"[altamacresists, devinnunes]",...,0,2019-11-23 02:36:23-05:00,2019-11-23,02:36:23,2019,11,23,2,36,2019-11-23 02:36:00


In [318]:
impeachment.head(5)

,cashtags,conversation_id,created_at,date,geo,hashtags,id,likes_count,link,mentions,...,video,eastern_created_at,eastern_date,eastern_time,year,month,day,hour,minute,eastern_date_time
138,[],1198057403545079808,2019-11-23 07:47:42,2019-11-22,,"[#devinnunesisatraitor, #devinnunesgotcaught]",1198146089905340417,32,https://twitter.com/slack2thefuture/status/119...,[devinnunes],...,0,2019-11-23 02:47:42-05:00,2019-11-23,02:47:42,2019,11,23,2,47,2019-11-23 02:47:00
139,[],1198083851484094464,2019-11-23 07:44:24,2019-11-22,,[],1198145255909888000,139,https://twitter.com/natfinnonE/status/11981452...,"[oliverdarcy, thegarance, devinnunes]",...,0,2019-11-23 02:44:24-05:00,2019-11-23,02:44:24,2019,11,23,2,44,2019-11-23 02:44:00
140,[],1198057403545079808,2019-11-23 07:43:13,2019-11-22,,[],1198144957942419456,8,https://twitter.com/dana1981/status/1198144957...,"[devinnunes, gop]",...,1,2019-11-23 02:43:13-05:00,2019-11-23,02:43:13,2019,11,23,2,43,2019-11-23 02:43:00
141,[],1198144318323621888,2019-11-23 07:40:40,2019-11-22,,"[#karma, #devinnunesgotcaught, #devinnunesisat...",1198144318323621888,53,https://twitter.com/taradublinrocks/status/119...,[devinnunes],...,0,2019-11-23 02:40:40-05:00,2019-11-23,02:40:40,2019,11,23,2,40,2019-11-23 02:40:00
142,[],1198138097562611712,2019-11-23 07:36:23,2019-11-22,,[],1198143241431863297,2,https://twitter.com/taradublinrocks/status/119...,"[altamacresists, devinnunes]",...,0,2019-11-23 02:36:23-05:00,2019-11-23,02:36:23,2019,11,23,2,36,2019-11-23 02:36:00


---

In [213]:
users = friday.groupby(['username']).agg('size').reset_index(name='mentions')

In [214]:
users.sort_values(by='mentions',ascending=False)

,username,mentions
194,naveedajamali,23
246,taradublinrocks,16
272,w7voa,15
73,dcexaminer,12
237,sovernnation,11
190,moveon,11
216,ricksmithshow,10
266,tppatriots,10
69,davidcorndc,8
108,gop,7
